# Исследование надёжности заёмщиков

**Цель:** Получить практичесские навыки в предобработке данных: обрабатывать пропущенные данные, работать с типами данных, выделять леммы, категоризировать данные.

**План выполнения работы:**

- 1. Изучение общей информации о данных
- 1. Предобработка данных
     - 2.1 Заполнение пропущенных данных
     - 2.2 Замена типа данных
     - 2.3 Удаление дубликатов
     - 2.4 Лемматизация целей получения кредитов
     - 2.5 Категоризация данных
- 1. Исследование данных
     - 3.1 Зависимость между наличием детей и возвратом кредита в срок
     - 3.2 Зависимость между семейным положением и возвратом кредита в срок
     - 3.3 Зависимость между уровнем дохода и возвратом кредита в срок
     - 3.4 Влияние целей кредита на его возврат в срок
- 1. Общий вывод

**Описание Данных:**
Данные представляют собой таблицу со статистическиой информацией о платёжеспособности клиентов:

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В файле с данным содержится 21525 строк, 12 столбцов.
Типы данных: строковый, вещественный, целочисленный.
В столбцах days_employed, total_income есть пропущенные значения.
Объем данных порядка 2 МБ.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#Определение пропущенных значений:
print('Количество пропущенных значений:', data['days_employed'].isnull().sum())

#Избавление от отрицательных значений в столбце "days_employed":
data['days_employed'] = data['days_employed'].abs()

#Нахождение медианного значение по всему столбцу "days_employed":
median = data['days_employed'].median()

#Замена аномально больших значений дней трудового стажа на медианное значение столбца "days_employed" (условие замены - более 26000 дней трудового стажа):
data.loc[(data['days_employed'] > 26000), 'days_employed'] = median

#Временная замена пропущенных значений на 0 (для корректной работы функции median_income_type):
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

#Определение уникальных категорий занятости:
status_type = data['income_type'].unique()

#Создание функции для замены нулевых значений на медианные в соответствии с категорией занятости:
def median_income_type(item):
    
    income_type = data['income_type']
    median_replace = data[item]
    
    for element in data:
        median_days_employed = data.groupby('income_type')[item].median()

    for element in status_type:
        status = data.loc[income_type == element]
        data.loc[(income_type==element)&(median_replace==0),item]=median_days_employed[element].astype(int)
    return median_replace

median_income_type('days_employed')
median_income_type('total_income')
median_income_type('dob_years')

#Вывод первых 30 строк таблицы с обработанными пропущенными значениями:
data.head(30)

    

Количество пропущенных значений: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

    В столбцах "days_employed" (количество дней трудового стажа) и "total_income" (суммарный доход) встречаются пропуски данных (Nan). Пропущенные данные относятся к категории количественных и составляют порядка 10% выборки, поэтому их удаление может привести к снижению качества анализа. Возможно, пропуски связаны с тем, что заемщики не были трудоустроены оффициально и не могли подтвердить свои доходы банку на момент выгрузки данных. Но данный факт не означает, что доходов у них нет, поэтому замена пропущенных значений нулями также может привести к снижению качества анализа.
    Также в данных столбца "days_employed" встречается такая ошибка выгрузки информации, как наличие отрицательных значений количества дней трудового стажа, что не может соответствовать действительности, так как данная величина может быть больше либо равной нулю. Для приведения данных к их абсолютному значению использован метод abs(). 
    Также в таблице встречаются такие аномалии, как количество дней трудового стажа, которые не могут соответствовать действительности (в некоторых случаях при пересчете на дни получается порядка 1000 лет трудового стажа). Данные аномалии встречаются преимущественно для категорий занятости "пенсионер" и "безработный". Возможная причина возникновения экстримальных значений - ошибка выгрузки данных, связанная с тем, что на момент формирования информации описанные выше категории граждан не числились трудоустроенными. Осуществлена замена экстримальных значений на медианное значение всего столбца "days_employed". Для обработки остальных пропусков создана функция по замене пропусков на медианные значения в соответствии с категорией занятости.

### Замена типа данных

In [3]:
#Приведение значений столбца "days_employed" к целочисленному типу:
data['days_employed'] = data['days_employed'].astype(int)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Количество дней трудового стажа (столбец "days_employed") не может быть вещественным числом, поэтому выполнено приведение данных к целочисленному типу с помощью метода astype(), аргументом которого является тип int.

### Обработка дубликатов

In [4]:
#Определение наличия дубликатов:
print('Количество дубликатов:', data.duplicated().sum())

#Определение уникальных значений столбца "education":
print('Значения столбца "education":', data['education'].unique())

#Приведение значений столбца "education" к нижнему регистру:
data['education'] = data['education'].str.lower()

#Повторное определение уникальных значений столбца "education":
print('Значения столбца "education":', data['education'].unique())
print('--------------')

#Определение уникальных значений столбца "family_status":
print('Значения столбца "family_status":', data['family_status'].unique())

#Приведение значений столбца "family_status" к нижнему регистру:
data['family_status'] = data['family_status'].str.lower()

#Повторное определение уникальных значений столбца "education":
print('Значения столбца "family_status":', data['family_status'].unique())
print('--------------')

#Повторное определение количества дубликатов после приведения к нижнему регистру:
print('Количество дубликатов:', data.duplicated().sum())

#Удаление дубликатов:
data = data.drop_duplicates().reset_index(drop=True)

#Контрольное определение наличия дубликатов:
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 54
Значения столбца "education": ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Значения столбца "education": ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
--------------
Значения столбца "family_status": ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Значения столбца "family_status": ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
--------------
Количество дубликатов: 71
Количество дубликатов: 0


**Вывод**

Для поиска дубликатов использовадся метод duplicated(), для их удаления - drop_duplicates(). Одна из причин появления дубликатов - наличие в выборке значений столбцов "education" в верхнем регистре. Также возможная причина появления дубликатов - агрегирование данных из разных источников в одну таблицу.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

#Лемматизация значений столбца "purpose" и запись результата в новый столбец "purpose_lemm":
purpose_lemm = []
for item in data['purpose']:
    lemmas = "".join(m.lemmatize(item))
    purpose_lemm.append(lemmas)
data['lemm_purpose'] = pd.Series(purpose_lemm)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,покупка жилье\n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,дополнительный образование\n
4,0,2194,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,сыграть свадьба\n
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,покупка жилье\n
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,операция с жилье\n
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование\n
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,на проведение свадьба\n
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,покупка жилье для семья\n


**Вывод**

Для того, чтобы осуществить лемматизацию значений столбца "purpose" осуществлено подключение специальной бибилиотеки pymystem3. Далее в цикле с помощью метода lemmatize() проведено выделение леммы для каждого слова.

### Категоризация данных

In [6]:
#Создание функции, выполняющей категоризацию данных в соответствии с целями кредитования:

def category_purpose(purpose):
    for item in data['lemm_purpose']:
        if 'авто' in purpose:
            return 'автомобиль'
        elif 'недвиж' in purpose:
            return 'жилье'
        elif 'жил' in purpose:
            return 'жилье'
        elif 'свад' in purpose:
            return 'свадьба'
        else:
            return 'образование'
        
data['purpose_category'] = data['lemm_purpose'].apply(category_purpose)
print(data['purpose_category'].value_counts())
print('-----------')

#Создание функции, выполняющей категоризацию заемщиков по наличию/отсутствию детей
def category_children(children):
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
data['children_category'] = data['children'].apply(category_children)
print(data['children_category'].value_counts())
print('-----------')

#Создание функции, выполняющей категоризацию данных по доходам:
def category_total_incom(income):
    if 0 < income < 100000:
        return 'доход до 100000'
    if 100000 <= income < 300000:
        return 'доход 100000 до 300000'
    if 300000 <= income < 500000:
        return 'доход от 300000 до 500000'
    else:
        return 'доход более 500000'    
data['income_category'] = data['total_income'].apply(category_total_incom)
print(data['income_category'].value_counts())
print('-----------')

#Создание функции, выполняющей категоризацию данных по возрасту заемщика:
def category_dob_years(age):
    if 18 <= age < 35:
        return 'от 18 до 35 лет'
    if 35 <= age <= 64:
        return 'от 35 до 64 лет'
    else:
        return 'старше 64 лет'
data['dob_years_category'] = data['dob_years'].apply(category_dob_years)
print(data['dob_years_category'].value_counts())

жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64
-----------
нет детей    14138
есть дети     7316
Name: children_category, dtype: int64
-----------
доход 100000 до 300000       15509
доход до 100000               4463
доход от 300000 до 500000     1260
доход более 500000             222
Name: income_category, dtype: int64
-----------
от 35 до 64 лет    14592
от 18 до 35 лет     5967
старше 64 лет        895
Name: dob_years_category, dtype: int64


**Вывод**

Категоризация данных осуществлена по цели получения кредита, по наличию/отсутствию детей, по уровню дохода. С помощью лемматизации  целей получения кредита, проведенной выше, осуществлено выделение ключевых слов и объединение их в соответствующие категории.
Самая популярная цель получения кредита - приобритение жилья (возможно это связано с тем, что ипотечное кредитование осуществляется под более низкий процент и на более длительный срок, что является более доступным видом кредитования).
Клиенты без детей больше кредитуются, нежели клиенты, имеющие детей (возможно связано с тем, что у клиентов без детей меньше финансовой ответственности).
Клиенты с доходом от 100000 до 300000 в месяц берут кредиты значительно чаще, чем клиенты, доход которых ниже 100000. При этом клиенты с доходом более 500000 берут кредиты значительно реже. Возможная причина: категории граждан со средним доходом банки больше одобряют кредиты в силу платежеспособности заемщиков. Категории граждан с доходом более 500000, возможно, реже обращаются к услугам банка. 
Больше всего кредитов берут клиенты в возрасте от 35 до 64 лет, что объясняется, вероятнее всего, их лучшей платежеспособностью.


In [12]:
data['total_income'].quantile([0.25, 0.5, 0.75])

0.25    107623.856558
0.50    140215.942303
0.75    195820.925921
Name: total_income, dtype: float64

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
children_unique = data['children'].unique()
print('Возможные варианты значений столбца "children":', children_unique)

#В данных столбца "children" встречаются некорректные значения: отрицательные и экстримальные, поэтому необходимо их скорректировать
#Избавление от отрицательных значений в столбце "children" (вероятнее всего, произошла ошибка при выгрузке данных и добавился "-" к цифре 1):
data['children'] = data['children'].abs()
#Замена значения 20 на 2 (вероятнее всего, произошла ошибка при выгрузке данных и добавился 0 к цифре 2)
data.loc[data['children'] == 20, 'children'] = 2

#Создание сводной таблицы для определения вероятности невозврата кредита в срок в зависимости от наличия/отсутствия детей
pivot_children = data.pivot_table(index = ['children_category'], values = 'debt').round(3) * 100
pivot_children.head()



Возможные варианты значений столбца "children": [ 1  0  3  2 -1  4 20  5]


,debt
children_category,
есть дети,9.3
нет детей,7.5


**Вывод**

Дебиторская задолженность встречается чаще у заемщиков имеющих детей. Возможно, это связано с тем, что у семей, имеющих детей, больше материальных обязательств и не всегда есть возможность вернуть кредитные средства вовремя.

In [13]:
data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4855   445  0.091658
2          2128   202  0.094925
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#Создание сводной таблицы для определения вероятности невозврата кредита в срок в зависимости от семейного положения:
pivot_family_status = data.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')
pivot_family_status['ratio_%'] = round(pivot_family_status[1]/(pivot_family_status[0]+pivot_family_status[1]), 3) * 100
pivot_family_status.sort_values('ratio_%', ascending = False)

debt,0,1,ratio_%
family_status,,,
не женат / не замужем,2536,274,9.8
гражданский брак,3763,388,9.3
женат / замужем,11408,931,7.5
в разводе,1110,85,7.1
вдовец / вдова,896,63,6.6


**Вывод**

Категории заемщиков, у которых семейное положение не женат/не замужем и гражданский брак имеют большую вероятность невозврата кредита в срок. Возможно, это связано с тем, что люди, находящиеся (находившиеся) в браке лучше умеют распоряжаться своими финансовыми ресурсами.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
pivot_income = data.pivot_table(index=['income_category'], columns='debt', values='gender', aggfunc='count')
pivot_income['ratio_%'] = round(pivot_income[1]/(pivot_income[0]+pivot_income[1]), 3) * 100
pivot_income.sort_values('ratio_%', ascending = False)

debt,0,1,ratio_%
income_category,,,
доход 100000 до 300000,14228,1281,8.3
доход до 100000,4109,354,7.9
доход от 300000 до 500000,1168,92,7.3
доход более 500000,208,14,6.3


**Вывод**

Клиенты с уровнем дохода более 500000 лучше возвращают кредиты в срок. Однако, клиенты с уровнем дохода от 100000 до 300000 имеют большую вероятность невозврата кредита в срок, чем клиенты с уровнем дохода до 100000. Возможно, это связано с тем, что у категории клиентов с уровнем дохода от 100000 до 300000 больше финансовых обязательств и трат, которые они не всегда способны исполнить.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
pivot_purpose = data.pivot_table(index=['purpose_category'], columns='debt', values='gender', aggfunc='count')
pivot_purpose['ratio_%'] = round(pivot_purpose[1]/(pivot_purpose[0]+pivot_purpose[1]), 3) * 100
pivot_purpose.sort_values('ratio_%', ascending = False)

debt,0,1,ratio_%
purpose_category,,,
автомобиль,3903,403,9.4
образование,3643,370,9.2
свадьба,2138,186,8.0
жилье,10029,782,7.2


**Вывод**

Самая большая вероятность невозврата кредита в срок - при покупке автомобиля, самая меньшая вероятность -  при покупке недвижимого имущества. Возможно, не всегда клиент, взявший кредитные средства на приобретение автомобиля, может содержать его. Также, возможно, что недвижимое имущество менее ценно с точки зрения психологии человека, чем приобретение жилья в собственность.

## Шаг 4. Общий вывод

На основании проведенного анализа можно сделать следующие выводы:
- повышенный скоринг у следующего клиента: человек, приобретающий в собственность недвижимость, с доходом не менее 300000, который состоит или состоял в браке и не имеет детей. 
- пониженный скоринг у следующего клиента: берет кредит с целью покупки автомобиля или на образовательные цели, с доходом от 100000 до 300000, не состоящий в браке, при этом имеющий на иждивении детей.